# Language Translation With Torchtext

This tutorial show how to use several convenience classes of `torchtext` to preprocess data from a well-known dataset containing sentence in both English and German and use it to train a sequence-to-sequence model with attention that can translate German sentences into English.

Be the end of this tutorial, you will be able to:

* Proprocess sentences into a commonly-used format for NLP modeling using the following `torchtext`
    * [TranslationDataset](https://torchtext.readthedocs.io/en/latest/datasets.html#torchtext.datasets.TranslationDataset)
    * [Field](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.Field)
    * [BucketIterator](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.BucketIterator)

## Field and TranslationDataset

`torchtext` has utilities for creating dataset that can be easily iterated through for the purpose of creating a language translation model. One key class is a `Field`, which specifies the way each sentence should be preprocessed, and another is the TranslationDataset; `torchtext` has several such datasets; in thisi tutorial we'll use the [Multi30k dataset](https://github.com/multi30k/dataset), which contains about 30,000 sentences in both English and German.

Note: the tokenization in this tutorial requires [Spacy](https://spacy.io/) We use Spacy because it provides strong support for tokenization in languages other than English. `torchtext` provides a `basic_english` tokenizer and supports other tokenizers for English but for language translation - where multiple languages are required - Spacy is your best bet.

With Spacy installed, the following code will tokenize each of the sentences in the `TranslationDataset` based on the tokenizer defined in the `Field`

In [46]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize='spacy', tokenizer_language='de',
           init_token='<sos>', eos_token='<eos>',
           lower=True)

TRG = Field(tokenize='spacy', tokenizer_language='en',
           init_token='<sos>', eos_token='<eos>',
           lower=True)

train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'),
                                                   fields=(SRC, TRG))

Now that we've defined `train_data`, we can see an extremely useful feature `torchtext`'s `Field`: the `build_vocab` method now allows us to create the vocabulary associated with each language.

In [47]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

Once these lines of code have been run, `SRC.vocab.stoi` will be a dictionary with the tokens in the vocabulary as keys and their corresponding indicies as values; `SRC.vocab.itos` will be the same dictionary with the keys and values swapped. We won't make extensive use of this fact in this tutorial, but this will likely be useful in other NLP tasks you'll encounter.

## BucketIterator

The last `torchtext` specific feature we'll use is the `BucketIterator`, which is easy to use since it takes a `TranslationDataset` as its first argument. Specifically, as the doc say: Defines an iterator that batches examples of similar lengths together. Minimizes amount of padding needed while producing shuffled batches for each new epoch. See pool for the bucketing procedure used.

In [48]:
import torch

device = torch.device('cpu')
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE)

## Defining our `nn.Module` and `Optimizer`

That's mostly it from a `torchtext` perspecive: with the dataset built and the iterator defined, the rest of this tutorail simply defines our model as an `nn.Module`, along with an `Optimizer`, and then trains it.

Note: this model is just an example model that can be used for language translation; we choose it because it is a standard model for the task, not because it is the recommended model to use for translation. As you're likely aware, state-of-the-art based on Transformers; you can see PyTorch's capacibilities for implementing Transformer layers [here](https://pytorch.org/docs/stable/nn.html#transformer-layers), and in particular, the "attention" used in the model below is different from the the multi-head self-attention present in a transformer model.

In [69]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    
    def __init__(self, input_dim, emb_dim, enc_hid_dim,
                dec_hid_dim, dropout):
        """
        :param input_dim: the vocabulary size
        :param emb_dim: embedding dimension
        :param enc_hid_dim: the encoder hidden dimension
        :param dec_hid_dim: decoder hidden dimension
        :param dropout: dropout
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim*2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src: Tensor) -> Tuple[Tensor]:
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:],
                                              hidden[-1,:,:]),
                                             dim=1)))
        return outputs, hidden


class Attention(nn.Module):
    
    def __init__(self, enc_hid_dim, dec_hid_dim, attn_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.attn_in = (enc_hid_dim*2) + dec_hid_dim
        self.attn = nn.Linear(self.attn_in, attn_dim)
        
    def forward(self, decoder_hidden, encoder_outputs) -> Tensor:
        src_len = encoder_outputs.shape[0]
        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((repeated_decoder_hidden,
                                                encoder_outputs),
                                                dim=2)))
        attention = torch.sum(energy, dim=2)
        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    
    def __init__(self, output_dim, emb_dim, enc_hid_dim,
                 dec_hid_dim, dropout, attention: nn.Module):
        super().__init__()
        
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.droptout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim*2) + emb_dim, dec_hid_dim)
        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def _weighted_encoder_rep(self, decoder_hidden: Tensor,
                             encoder_outputs: Tensor) -> Tensor:
        a = self.attention(decoder_hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)
        return weighted_encoder_rep
    
    def forward(self, input, decoder_hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                         encoder_outputs)
        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim=2)
        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)
        output = self.out(torch.cat((output, weighted_encoder_rep,
                                    embedded), dim=1))
        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio:float=0.5):
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vacab_size = self.decoder.output_dim
    
        outputs = torch.zeros(max_len, batch_size, trg_vacab_size)
        
        encoder_ouputs, hidden = self.encoder(src)
        
        # first input to the decoder is the <sos> token
        output = trg[0, :]
        
        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_ouputs)
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)
            
        return outputs

    
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROOPOUT = 0.5


enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, 
              DEC_HID_DIM, ENC_DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, 
              DEC_HID_DIM, DEC_DROOPOUT, attn)
model = Seq2Seq(enc, dec, device)


def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            

model.apply(init_weights)
optimizer = optim.Adam(model.parameters())


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,856,685 trainable parameters


Note: when scoring the performance of a language translation model in particular, we have to tell the `nn.CrossEntropyLoss` function to ignore the indices where the target is simply padding.

In [60]:
PAD_IDX = TRG.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

Finally, we can train and evaluate this model:

In [ ]:
import math
import time

def train(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    
    for _, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0) # trun off teacher forcing
            output = output[1:].view(-1, ouptput.shape[-1])
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer,
                      criterion, CLIP)
    valid_loss = evaluate(mode, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    test_loss = evaluate(model, test_iterator, criterion)
    
    print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')
